In [25]:
import sqlite3
import pandas as pd
import os
import datetime as datetime
import numpy as np
from dotenv import dotenv_values
from binance.client import Client
import requests
import random
import yfinance as yf
import sys 
import math

In [26]:
is_auto=False

no_last_items=1
no_set=5
x_now=datetime.datetime.now()

api_source='yahoo'
#api_source='bn'

In [27]:
if is_auto==True:
    if len(sys.argv)==2:
       print(sys.argv[1] ) 
       api_source=sys.argv[1] 
        
       with open('readme.txt', 'w') as f:
        for i in range(len(sys.argv)):
         f.write(sys.argv[i]+"\n") 
    else:
        raise Exception("take api_source as param bn or yahoo")


#exit()


In [28]:
config_path=r'D:\AB_DB\Script_ImportData\.env'
data_base_file='alert_price_db.sqlite3'
# data_base_file=r'D:\AB_DB\Script_ImportData\Database\alert_price_db.sqlite3'

In [29]:
buildtime = x_now.strftime('%d%b%y_%H%M')
print(buildtime)

12Jul23_0053


In [30]:
# config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
config_values= dotenv_values(dotenv_path=config_path)
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)

line_token = config_values['line_token']

In [31]:
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(data_base_file))

sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

Connect SQLite


In [32]:
if api_source=='bn':
    sql_price_range="select * from symbol_price_info where is_active=1 and api_source='bn'"

elif api_source=='yahoo':
    sql_price_range="select * from symbol_price_info where is_active=1 and api_source='yahoo'"
    
df=pd.read_sql_query(sql_price_range, conn)
df['symbol_pair']=df.apply( lambda x: f"{x.symbol_x}{x.symbol_pair}",axis=1)

print(df)



  symbol_key symbol_x symbol_pair  price_range  is_active api_source
0     s&p500      SPY      SPYUSD  400,420,335          1      yahoo
1      nadaq      QQQ      QQQUSD  360,370,380          1      yahoo


In [33]:
# get price from binace

# get price from yahoo
# https://aroussi.com/post/python-yahoo-finance
# https://github.com/ranaroussi/yfinance

current_price_dict={}
list_symbol=[]

if api_source=='bn':
    for index,item in df.iterrows():
        current_price =(client.get_symbol_ticker(symbol=item["symbol_pair"]))['price']
        # current_price=random.randint(1, 30)
        current_price_dict[item["symbol_key"]]= round( float(current_price),4)
        list_symbol.append(item["symbol_key"])
          
        
#auto_adjust = True
elif api_source=='yahoo':
    fund_symbols=df['symbol_x'].tolist()
    fund_keys=df['symbol_key'].tolist()
    
    symbol_str=' '.join(fund_symbols )
    print(symbol_str)
    list_current_price = yf.download( tickers = symbol_str,threads = True,
                                 group_by=True,period = "1d",prepost = True)
    print(list_current_price)
    list_fund= list(zip(fund_keys,fund_symbols))
    for x in list_fund:
        # print(type(list_current_price))
        # if len(list_current_price)==1:
        #  print("xyx")   
        #  current_price_dict[x[0]]=round( list_current_price['Close'][0],2) # new verion
        # else:
         # print("abc")   
        current_price_dict[x[0]]=round( list_current_price[x[1]]['Close'][0],2) # old verion 2022
        list_symbol.append(x[0])

print(current_price_dict )  


SPY QQQ
[*********************100%***********************]  2 of 2 completed
                   QQQ                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2023-07-11  366.790009  367.945007  364.619995  366.582703  366.582703   

                             SPY                                      \
              Volume        Open        High         Low       Close   
Date                                                                   
2023-07-11  25460441  440.450012  441.700012  439.440002  440.834991   

                                  
             Adj Close    Volume  
Date                              
2023-07-11  440.834991  30734998  
{'s&p500': 440.83, 'nadaq': 366.58}


In [34]:
def get_last_record():
    conn = sqlite3.connect(os.path.abspath(data_base_file))

    df_all_last_update=pd.DataFrame()

    for item in list_symbol:
        sql=f"select * from notification_transaction where symbol_key='{item}' order by datetime desc limit 1"
        df_item=pd.read_sql_query(sql, conn)
        df_all_last_update=pd.concat([df_all_last_update,df_item])
        #print(df_item)
    return df_all_last_update


In [35]:
df_all_prev=get_last_record()
df_all_prev

,datetime,symbol_key,price,all_ranges,no_ranges,idx_pos,idx_pct,msg,prev_msg,prev_pct_chg,prev_price
0,2023-07-12 00:38:44,s&p500,440.87,"400,420,335",4,4,100.0,s&p500:440.87 > 420.0,None,None,None
0,2023-07-12 00:38:44,nadaq,366.61,"360,370,380",4,2,50.0,360.0 <= (366.61) >= 370.0,None,None,None


In [36]:
print(list_symbol)

['s&p500', 'nadaq']


In [37]:
dict_prev_item={}
for  symb in  list_symbol:
    df_temp=df_all_prev.query("symbol_key==@symb")
    #print(df_temp)
    str_prev_msg=None
    prev_price=None
    # print(df_temp)
    if  df_temp.empty==False:
      temp=df_temp.loc[0]
      str_prev_msg=f"{temp['idx_pos']}# {temp['msg']}"
      prev_price=  temp['price']

    dict_prev_item[symb]=(str_prev_msg,prev_price)
print(dict_prev_item)

{'s&p500': ('4# s&p500:440.87 > 420.0', 440.87), 'nadaq': ('2# 360.0 <= (366.61) >= 370.0', 366.61)}


In [38]:
def find_range(item):
   symbol_key=item['symbol_key'] 

   p=current_price_dict[symbol_key]
   print(f'{symbol_key} : {p}') 

   priceList_str= item["price_range"].strip().split(",")
   priceList=[ float(x) for x in priceList_str]
   list.sort(priceList) 
   print(priceList)
     
   log_info={}

   log_info['datetime']=(x_now).strftime("%Y-%m-%d %H:%M:%S")
   log_info['symbol_key']=symbol_key   
   log_info['price']=p
   log_info['all_ranges']= item["price_range"]
   
   range_length=len(priceList)
   log_info['no_ranges']=range_length+1
   
   position=0;
   for index in range(0,range_length):   
     # print("iterate:",index)
     if (index==0) and (p<priceList[index]):
            position= 1
            log_info['msg']=f"{symbol_key}:{p} < {priceList[index]}"
            
     elif (index>0) and (index< range_length ) and ( p>=priceList[index-1] and p<=priceList[index] ):
            position=  index+1
            log_info['msg']= f"{priceList[index-1]} <= ({p}) >= {priceList[index] }"
          
     elif   (index==range_length-1) and (p>priceList[index])    :
            position= range_length+1
            log_info['msg']= f"{symbol_key}:{p} > {priceList[index]}"
   
   print( log_info['msg'])   
    
   # write prev log info         
   if  symbol_key in dict_prev_item:

        msg,prev_p=dict_prev_item[symbol_key]

        if msg is not None and prev_p is not None :
         log_info['prev_msg']=msg   
         log_info['prev_price']= prev_p
         log_info['prev_pct_chg']= round( (p-prev_p)/prev_p*100 ,1)
        else:
         log_info['prev_msg']=None   
         log_info['prev_price']=None   
         log_info['prev_pct_chg']=None
         
    
   log_info['idx_pos']=position 
   log_info['idx_pct']= round( log_info['idx_pos']/log_info['no_ranges']*100,1)  
                
   # else:
   #      raise Exception("There are someting wrong!!!!")
        
   print(log_info)
   print("===========================================================")
   return log_info 
   
df['transaction']=df.apply(find_range,axis=1)  

s&p500 : 440.83
[335.0, 400.0, 420.0]
s&p500:440.83 > 420.0
{'datetime': '2023-07-12 00:53:58', 'symbol_key': 's&p500', 'price': 440.83, 'all_ranges': '400,420,335', 'no_ranges': 4, 'msg': 's&p500:440.83 > 420.0', 'prev_msg': '4# s&p500:440.87 > 420.0', 'prev_price': 440.87, 'prev_pct_chg': -0.0, 'idx_pos': 4, 'idx_pct': 100.0}
nadaq : 366.58
[360.0, 370.0, 380.0]
360.0 <= (366.58) >= 370.0
{'datetime': '2023-07-12 00:53:58', 'symbol_key': 'nadaq', 'price': 366.58, 'all_ranges': '360,370,380', 'no_ranges': 4, 'msg': '360.0 <= (366.58) >= 370.0', 'prev_msg': '2# 360.0 <= (366.61) >= 370.0', 'prev_price': 366.61, 'prev_pct_chg': -0.0, 'idx_pos': 2, 'idx_pct': 50.0}


In [39]:
print(df)

  symbol_key symbol_x symbol_pair  price_range  is_active api_source  \
0     s&p500      SPY      SPYUSD  400,420,335          1      yahoo   
1      nadaq      QQQ      QQQUSD  360,370,380          1      yahoo   

                                         transaction  
0  {'datetime': '2023-07-12 00:53:58', 'symbol_ke...  
1  {'datetime': '2023-07-12 00:53:58', 'symbol_ke...  


In [40]:
list_trans=[]
for index,item in df.iterrows():
    list_trans.append(item['transaction'])
df_tran=pd.DataFrame(list_trans)

df_tran['idx_pos']=df_tran['idx_pos'].astype('int')


print(df_tran.info())
df_tran

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      2 non-null      object 
 1   symbol_key    2 non-null      object 
 2   price         2 non-null      float64
 3   all_ranges    2 non-null      object 
 4   no_ranges     2 non-null      int64  
 5   msg           2 non-null      object 
 6   prev_msg      2 non-null      object 
 7   prev_price    2 non-null      float64
 8   prev_pct_chg  2 non-null      float64
 9   idx_pos       2 non-null      int32  
 10  idx_pct       2 non-null      float64
dtypes: float64(4), int32(1), int64(1), object(5)
memory usage: 296.0+ bytes
None


,datetime,symbol_key,price,all_ranges,no_ranges,msg,prev_msg,prev_price,prev_pct_chg,idx_pos,idx_pct
0,2023-07-12 00:53:58,s&p500,440.83,"400,420,335",4,s&p500:440.83 > 420.0,4# s&p500:440.87 > 420.0,440.87,-0.0,4,100.0
1,2023-07-12 00:53:58,nadaq,366.58,"360,370,380",4,360.0 <= (366.58) >= 370.0,2# 360.0 <= (366.61) >= 370.0,366.61,-0.0,2,50.0


In [42]:
def insertMultipleItems(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_query = """
        INSERT INTO notification_transaction
        (datetime, symbol_key,price,all_ranges,no_ranges,
         msg,prev_msg,prev_price,prev_pct_chg,idx_pos,idx_pct) 
         VALUES (?,?,?, ?, ?,?,?,?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        sqliteConnection.commit()
        print("Total", cursor.rowcount, "Add transaction successfully")
        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert multiple records into product table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")


recordsToInsert=list(df_tran.to_records(index=False))
insertMultipleItems(recordsToInsert)

Connected to SQLite
Total 2 Add transaction successfully
The SQLite connection is closed


In [43]:
df_all_last_update=get_last_record()

#df_all_last_update=df_all_last_update[['datetime','symbol_key','price','prev_price','prev_pct_chg','msg','prev_msg','idx_pct','idx_pos','all_ranges']]
df_all_last_update=df_all_last_update[['datetime','symbol_key','price','prev_price','prev_pct_chg','msg','idx_pos','all_ranges']]
df_all_last_update.fillna("-",inplace=True)
df_all_last_update.info()
df_all_last_update

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      2 non-null      object 
 1   symbol_key    2 non-null      object 
 2   price         2 non-null      float64
 3   prev_price    2 non-null      float64
 4   prev_pct_chg  2 non-null      float64
 5   msg           2 non-null      object 
 6   idx_pos       2 non-null      int64  
 7   all_ranges    2 non-null      object 
dtypes: float64(3), int64(1), object(4)
memory usage: 144.0+ bytes


,datetime,symbol_key,price,prev_price,prev_pct_chg,msg,idx_pos,all_ranges
0,2023-07-12 00:53:58,s&p500,440.83,440.87,0.0,s&p500:440.83 > 420.0,4,"400,420,335"
0,2023-07-12 00:53:58,nadaq,366.58,366.61,0.0,360.0 <= (366.58) >= 370.0,2,"360,370,380"


In [44]:
#!/usr/local/bin/python3
import requests
url = 'https://notify-api.line.me/api/notify'
token = line_token
headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}

In [45]:
no_split=math.ceil(len(df_all_last_update)/no_set)
print(no_split)
info_array = np.array_split(df_all_last_update,no_split)
for info in info_array:
    string_msg=info.to_string(index=False,header=True)
    string_msg='\n'+string_msg
    print(len(string_msg))
    r = requests.post(url, headers=headers, data = {'message':string_msg})
    print (r.text)
    print("===========================================") 

1
336
{"status":200,"message":"ok"}
